In [ ]:
sig_correct_path = "../sig_correct.txt"
secrets_path = "../secrets.txt"
logfile = "hammer.log"

In [ ]:
def analyze_faulty_sig(sig_correct, sig_faulty) -> str:
    # Analyze faulty sig
    sig_field_widths = [('H(m)', 32), ('FORS', 10560)] + [(f'WOTS{i}', 2400) for i in range(8)] + [('msg', 0)]
    i = 0
    f = None
    for cf, cc in zip(sig_faulty, sig_correct):
        if cf != cc:
            print(f"Bit {i} differs: {cf} != {cc}")
            for field, width in sig_field_widths:
                if i < width:
                    print(f"Bit {i} is in field {field}")
                    f = field
                    return f
                # print(f"{i} -= {width}")
                i -= width
            else:
                print(f"Bit {i} is in field msg")
                return "msg"
        i += 1/2
    return f

In [ ]:
sigs = "../sigs.txt"
#sig_correct = open(sig_correct_path).readline().strip()

from collections import Counter

with open(sigs) as f:
    lines = f.readlines()
    sig_correct = [line for line in lines if line.startswith("[CORRECT]")]
    assert len(sig_correct) >= 1, "There should be at least one correct signature"
    sig_correct = sig_correct[0][len("[CORRECT] "):].strip()
    faulty = [line[len("[FAULTY] "):].strip() for line in lines if line.startswith("[FAULTY]")]
    lines = Counter(faulty)
    for line in lines:
        sig = line.strip()
        assert sig != sig_correct, "Signature should not be correct"
        import postprocessing as pp
        import hashlib
        is_expl = pp.is_exploitable(sig_correct, sig, secrets_path)
        sha256_hash = hashlib.sha256(sig.encode()).hexdigest()
        print(f"{sha256_hash} ({lines[line]}x) exploitable: {is_expl}")
        if is_expl:
            analyze_faulty_sig(sig_correct, sig)

In [ ]:
import glob
import pandas as pd
import zipfile


# Get all exploitable_*.zip files
zip_files = glob.glob('results/*.zip')

candidates = []
for zip_filename in zip_files:
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        with zipf.open('sig_correct.txt') as sig_correct_file:
            sig_correct = sig_correct_file.readline().decode('utf-8').strip()
        with zipf.open('sig_faulty.txt') as sig_faulty_file:
            sig_faulty = sig_faulty_file.readline().decode('utf-8').strip()
    exploitable = pp.is_exploitable(sig_correct, sig_faulty, secrets_path)
    common_prefix_length = pp.prefix(sig_correct, sig_faulty)
    candidates.append([zip_filename, common_prefix_length, len(sig_faulty), len(sig_correct), exploitable])
    
candidates = sorted(candidates, key=lambda x: x[1], reverse=True)

dataset = pd.DataFrame(candidates, columns = ['ID', 'common_prefix_length', 'Faulty len', 'Correct len', 'is_exploitable'])
    
print(dataset)